# Testing

## News Scraper

Probably need to add data cleaning.

In [ ]:
# pip install newspaper3k
# pip install lxml[html_clean]

In [ ]:
import pandas as pd

Import scraping function `scrape_to_csv` from `scrape.py`.

In [2]:
from scrape import scrape_to_csv

In [9]:
url_list = [
    "https://globalnews.ca/news/10863638/donald-trump-tom-homan-border-czar/",
    "https://www.ctvnews.ca/world/trump-names-stephen-miller-to-be-deputy-chief-of-policy-in-new-administration-1.7105734",
    "https://www.bbc.com/news/articles/cy4g0vg8gdqo",
    "https://www.forbes.com/sites/saradorn/2024/11/11/trumps-cabinet-heres-who-hes-picked-for-top-jobs-as-stephen-miller-chosen-for-white-house-role/",
    "https://www.cnn.com/2024/11/11/politics/stephen-miller-trump-administration/index.html",
    "https://www.bbc.com/news/articles/c3vlndv0yxpo",
    "https://www.washingtonpost.com/national-security/2024/11/10/trump-putin-phone-call-ukraine/",
    "https://www.bbc.com/news/articles/cy4g0vg8gdqo",
    "https://www.cbc.ca/news/world/trump-putin-phone-call-ukraine-1.7380033"
]
category = "Politics"
filename = "news.csv"

In [10]:
for url in url_list:
    scrape_to_csv(url, category, filename)

Duplicate article found based on URL. Skipping...
Duplicate article found based on URL. Skipping...
Duplicate article found based on URL. Skipping...
Failed to download or parse article: Article `download()` failed with 403 Client Error: Max restarts limit reached for url: https://www.forbes.com/sites/saradorn/2024/11/11/trumps-cabinet-heres-who-hes-picked-for-top-jobs-as-stephen-miller-chosen-for-white-house-role/ on URL https://www.forbes.com/sites/saradorn/2024/11/11/trumps-cabinet-heres-who-hes-picked-for-top-jobs-as-stephen-miller-chosen-for-white-house-role/
Article 'Trump expected to announce Stephen Miller as deputy chief of staff for policy' saved to news.csv.
Article 'Who is Elise Stefanik, Trump's pick for UN ambassador?' saved to news.csv.
Article 'Trump talked to Putin, told Russian leader not to escalate in Ukraine' saved to news.csv.
Duplicate article found based on URL. Skipping...
Article 'Source says Trump advised Putin not to escalate Ukraine war, Kremlin denies conv

In [3]:
filename = "news.csv"
df = pd.read_csv(filename)

In [4]:
df.head(1)

,ID,Date of Article,Category,Title,Body,URL
0,1731361091,2024-11-11,Politics,Donald Trump taps former ICE director Tom Homa...,Send this page to someone via email\n\nPreside...,https://globalnews.ca/news/10863638/donald-tru...


In [5]:
df['Title']

0    Donald Trump taps former ICE director Tom Homa...
1    Trump names Stephen Miller to be deputy chief ...
2    Russia denies Trump call with Putin urging res...
3    Trump expected to announce Stephen Miller as d...
4    Who is Elise Stefanik, Trump's pick for UN amb...
5    Trump talked to Putin, told Russian leader not...
6    Source says Trump advised Putin not to escalat...
Name: Title, dtype: object

## Agents

In [12]:
df_summary = df.copy()

In [15]:
df_summary['Zero-Shot Summary'] = pd.NA

Import agents from `agent.py`.

### Zero-Shot Summary

In [13]:
from agent import SummarizationAgent

In [14]:
summarization_agent = SummarizationAgent()

Generate Zero-shot summaries.

In [16]:
def generate_summary(body):
    try:
        return summarization_agent.process(body)
    except Exception as e:
        print(f"Failed to generate summary: {e}")
        return pd.NA

In [ ]:
df_summary['Zero-Shot Summary'] = df_summary['Body'].apply(generate_summary)

2024-11-11 19:54:54,959 - Summarization Agent - INFO - Starting summarization...
2024-11-11 19:54:54,960 - Summarization Agent - DEBUG - Generating initial summary...
2024-11-11 19:54:57,742 - Summarization Agent - INFO - Received response: President-elect Donald Trump announced that Tom Homan, former acting director of U.S. Immigration and Customs Enforcement (ICE), will serve as "border czar" in his administration, responsible for overseeing the country's borders and deportation operations. Homan's role includes managing security at borders, maritime, and aviation points, as well as handling the deportation of illegal immigrants, which is central to Trump's agenda. Homan assured that these operations would be conducted humanely and strategically by ICE, emphasizing the efficiency of the organization's personnel in executing such tasks.
2024-11-11 19:54:57,742 - Summarization Agent - INFO - Summary generated successfully.
2024-11-11 19:54:57,743 - Summarization Agent - INFO - Starting

In [ ]:
new_file = "summaries.csv"

df_summary.to_csv(new_file, index=False)
print(f"Zero-Shot Summaries saved to {new_file}")

Zero-Shot Summaries saved to summaries.csv


In [21]:
new_file = "summaries.csv"
summaries = pd.read_csv(new_file)

In [22]:
summaries.head(2)

,ID,Date of Article,Category,Title,Body,URL,Zero-Shot Summary
0,1731361091,2024-11-11,Politics,Donald Trump taps former ICE director Tom Homa...,Send this page to someone via email\n\nPreside...,https://globalnews.ca/news/10863638/donald-tru...,President-elect Donald Trump announced that To...
1,1731361091,2024-11-11,Politics,Trump names Stephen Miller to be deputy chief ...,NEW YORK -\n\nU.S. President-elect Donald Trum...,https://www.ctvnews.ca/world/trump-names-steph...,U.S. President-elect Donald Trump has appointe...


### Personalization

How do we actually get the background? (Survey, reading history?, ideally with usage)

In [ ]:
background = ["Donald Trump", "2022 Ukraine War", "2024 US election"]

`PersonalizationAgent` generates **persona(s)** and **feedback**.

- **Persona(s)**
    - Get from background, keywords
    - Obtained from:
        - Survey
        - Read history
        - Usage
- **Feedback**
    - Feedback based on persona and passed back to `SummarizationAgent`

In [ ]:
from agent import PersonalizationAgent
personalization_agent = PersonalizationAgent(background)